In [ ]:
#나의 직접구현

In [45]:
import pandas as pd
import numpy as np
input_file=pd.read_csv("C:/Users/kang9/Downloads/naivebayes_example.csv")
input_file

,mail,label
0,i love you,1
1,love happy weekend,1
2,bore work job,0
3,i hate you,0
4,bore weekend,0
5,happy together,1


In [26]:
docs=(input_file['mail'])

In [27]:
docs

0            i love you
1    love happy weekend
2         bore work job
3            i hate you
4          bore weekend
5        happy together
Name: mail, dtype: object

In [28]:
doc_ls=[]
for doc in docs:
    doc_ls.append(doc.split())
doc_ls

[['i', 'love', 'you'],
 ['love', 'happy', 'weekend'],
 ['bore', 'work', 'job'],
 ['i', 'hate', 'you'],
 ['bore', 'weekend'],
 ['happy', 'together']]

In [29]:
#유니크한 것 토큰화
from collections import defaultdict
word2id = defaultdict(lambda:len(word2id))

#중복이 되는 토큰은 한번씩만 들어가게 된다.
for doc in doc_ls:
    for token in doc:
        word2id[token]  #토큰을 인덱스로
word2id

defaultdict(<function __main__.<lambda>()>,
            {'i': 0,
             'love': 1,
             'you': 2,
             'happy': 3,
             'weekend': 4,
             'bore': 5,
             'work': 6,
             'job': 7,
             'hate': 8,
             'together': 9})

In [30]:
positive=[]
for i,word in enumerate(doc_ls):
    for po in word:
        if input_file['label'][i]==1:
            positive.append(po)

In [31]:
positive

['i', 'love', 'you', 'love', 'happy', 'weekend', 'happy', 'together']

In [32]:
negative=[]
for i,word in enumerate(doc_ls):
    for ne in word:
        if input_file['label'][i]==0:
            negative.append(ne)

In [33]:
negative

['bore', 'work', 'job', 'i', 'hate', 'you', 'bore', 'weekend']

In [34]:
po_la=np.zeros(len(word2id),dtype=int)

for token in positive:
    po_la[word2id[token]] +=1 #해당토큰위치(column)
 
print(po_la)

[1 2 1 2 1 0 0 0 0 1]


In [35]:
ne_la=np.zeros(len(word2id),dtype=int)

for token in negative:
    ne_la[word2id[token]] +=1 #해당토큰위치(column)
 
print(ne_la)

[1 0 1 0 1 2 1 1 1 0]


In [36]:
df=pd.DataFrame(index=word2id)
df

""
i
love
you
happy
weekend
bore
work
job
hate
together


In [37]:
df['positive']=po_la
df

,positive
i,1
love,2
you,1
happy,2
weekend,1
bore,0
work,0
job,0
hate,0
together,1


In [38]:
df['nagative']=ne_la
df

,positive,nagative
i,1,1
love,2,0
you,1,1
happy,2,0
weekend,1,1
bore,0,2
work,0,1
job,0,1
hate,0,1
together,1,0


In [ ]:
#선생님 직접구현

In [16]:
import pandas as pd
import numpy as np
input_file=pd.read_csv("C:/Users/kang9/Downloads/naivebayes_example.csv")

In [17]:
training_set = np.array(input_file)
print(training_set)

[['i love you' 1]
 ['love happy weekend' 1]
 ['bore work job' 0]
 ['i hate you' 0]
 ['bore weekend' 0]
 ['happy together' 1]]


In [18]:
training_set

array([['i love you', 1],
       ['love happy weekend', 1],
       ['bore work job', 0],
       ['i hate you', 0],
       ['bore weekend', 0],
       ['happy together', 1]], dtype=object)

In [19]:
# 토큰별로 문서 내 빈도수 카운팅
from collections import defaultdict
wordfreq = defaultdict(lambda : [0,0])  # lambda : 새로운 단어가 추가 될때 default 를 [0,0]으로 하겠다. [긍정,부정]
for doc, point in training_set:
    words = doc.split()
    for word in words:
        if point == 1:   
            wordfreq[word][0] +=1 # 긍정인 경우 0번째 인텍스에 1더해라 
        else:
            wordfreq[word][1] +=1 # 부정인 경우 1번째 인텍스에 1더해라
        
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [20]:
# 긍정/ 부정 빈도수 계산
긍정전체토큰수=[]
부정전체토큰수=[]
for key, (cnt1,cnt0) in wordfreq.items():
    긍정전체토큰수.append(int(cnt1))
    부정전체토큰수.append(int(cnt0))
print(긍정전체토큰수)
print(부정전체토큰수)
전체갯수_긍정 = sum(긍정전체토큰수)
전체갯수_부정 = sum(부정전체토큰수)
print(전체갯수_긍정,전체갯수_부정)

[1, 2, 1, 2, 1, 0, 0, 0, 0, 1]
[1, 0, 1, 0, 1, 2, 1, 1, 1, 0]
8 8


In [21]:
wordfreq

defaultdict(<function __main__.<lambda>()>,
            {'i': [1, 1],
             'love': [2, 0],
             'you': [1, 1],
             'happy': [2, 0],
             'weekend': [1, 1],
             'bore': [0, 2],
             'work': [0, 1],
             'job': [0, 1],
             'hate': [0, 1],
             'together': [1, 0]})

In [22]:
wordprobs = defaultdict(lambda:[0,0])
for key, (cnt1,cnt0) in wordfreq.items():
    wordprobs[key][0] = (cnt1 + 1)/(전체갯수_긍정 + len(wordfreq))
    wordprobs[key][1] = (cnt0 + 1)/(전체갯수_부정 + len(wordfreq))
wordprobs

defaultdict(<function __main__.<lambda>()>,
            {'i': [0.1111111111111111, 0.1111111111111111],
             'love': [0.16666666666666666, 0.05555555555555555],
             'you': [0.1111111111111111, 0.1111111111111111],
             'happy': [0.16666666666666666, 0.05555555555555555],
             'weekend': [0.1111111111111111, 0.1111111111111111],
             'bore': [0.05555555555555555, 0.16666666666666666],
             'work': [0.05555555555555555, 0.1111111111111111],
             'job': [0.05555555555555555, 0.1111111111111111],
             'hate': [0.05555555555555555, 0.1111111111111111],
             'together': [0.1111111111111111, 0.05555555555555555]})

In [23]:
import math
doc = "happy weekend" 
tokens = doc.split()
tokens

['happy', 'weekend']

In [24]:
# 초기값을 모두 0으로 처리 
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복 
for word, (prob1,prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 +=math.log(prob1)
        log_prob0 +=math.log(prob0)
log_prob1 += math.log(전체갯수_긍정/(전체갯수_긍정+전체갯수_부정))
log_prob0 += math.log(전체갯수_부정/(전체갯수_긍정+전체갯수_부정))

prob1 = math.exp(log_prob1) 
print('prob1',prob1)
prob0 = math.exp(log_prob0)   
print('prob0',prob0)
print('happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률')
print("긍정확률 : {}%".format(prob1/(prob1+prob0)*100))
print("부정확률 : {}%".format(prob0/(prob1+prob0)*100))

prob1 0.009259259259259257
prob0 0.0030864197530864183
happy와 weekend가 새로운 메일에 포함되어 있을 경우, 긍정확률과 부정확률
긍정확률 : 75.00000000000001%
부정확률 : 24.999999999999996%


In [ ]:
#sklearn

In [28]:
X_train=list(input_file['mail'])
Y_train=list(input_file['label'])
print(X_train)
print(Y_train)

['i love you', 'love happy weekend', 'bore work job', 'i hate you', 'bore weekend', 'happy together']
[1, 1, 0, 0, 0, 1]


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
#CointVectorizer 선언
count_vect=CountVectorizer()
#fit and transform
X_train_count=count_vect.fit_transform(X_train)
#MultinomialNB
clf=MultinomialNB().fit(X_train_count,Y_train)

In [34]:
#예측
print(clf.predict(count_vect.transform(["happy weekend"])))
#확률
print(clf.predict_proba(count_vect.transform(["happy weekend"])))

[1]
[[0.25 0.75]]
